In [19]:
import tensorflow as tf
import os

In [48]:
settings = "polar-window20-lag0-resnet152-v2"
save_dir = "/ml-models/db16.4/"

In [49]:
import onnx

# Load the ONNX model
model = onnx.load(f"{settings}.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a Human readable representation of the graph
onnx.helper.printable_graph(model.graph)

'graph torch-jit-export (\n  %actual_input[FLOAT, 1x1x20x75]\n) initializers (\n  %decoder.decoder.weight[FLOAT, 2x2048]\n  %decoder.decoder.bias[FLOAT, 2]\n  %1404[FLOAT, 64x1x7x7]\n  %1405[FLOAT, 64]\n  %1407[FLOAT, 256x64x1x1]\n  %1408[FLOAT, 256]\n  %1410[FLOAT, 64x64x1x1]\n  %1411[FLOAT, 64]\n  %1413[FLOAT, 64x64x3x3]\n  %1414[FLOAT, 64]\n  %1416[FLOAT, 256x64x1x1]\n  %1417[FLOAT, 256]\n  %1419[FLOAT, 64x256x1x1]\n  %1420[FLOAT, 64]\n  %1422[FLOAT, 64x64x3x3]\n  %1423[FLOAT, 64]\n  %1425[FLOAT, 256x64x1x1]\n  %1426[FLOAT, 256]\n  %1428[FLOAT, 64x256x1x1]\n  %1429[FLOAT, 64]\n  %1431[FLOAT, 64x64x3x3]\n  %1432[FLOAT, 64]\n  %1434[FLOAT, 256x64x1x1]\n  %1435[FLOAT, 256]\n  %1437[FLOAT, 512x256x1x1]\n  %1438[FLOAT, 512]\n  %1440[FLOAT, 128x256x1x1]\n  %1441[FLOAT, 128]\n  %1443[FLOAT, 128x128x3x3]\n  %1444[FLOAT, 128]\n  %1446[FLOAT, 512x128x1x1]\n  %1447[FLOAT, 512]\n  %1449[FLOAT, 128x512x1x1]\n  %1450[FLOAT, 128]\n  %1452[FLOAT, 128x128x3x3]\n  %1453[FLOAT, 128]\n  %1455[FLOAT, 51

In [23]:
import onnx_coreml

ModuleNotFoundError: No module named 'coremltools.converters.nnssa'

In [19]:
def convert(model,
           mode="classifier",
           image_input_names=[],
           preprocessing_args={"window_size": 10, "lag": 200, "features_all": True},
           image_output_names=[],
           class_labels=["no-fall","fall"],
           predicted_feature_name="classLabel",
           add_custom_layers=False,
           custom_conversion_functions={},
           minimum_ios_deployment_target="13")

SyntaxError: invalid syntax (1828637152.py, line 10)

In [50]:
from onnx_tf.backend import prepare

tf_rep = prepare(model)

In [51]:
tf_model_path = os.getcwd() + save_dir + f"{settings}-tf/"
tflite_model_path = tf_model_path + f"saved_model.tflite"

In [52]:
tf_rep.export_graph(tf_model_path)

INFO:tensorflow:Assets written to: /home/hwixley/Documents/4th-Year/Honours-Project/localhost-data-preprocessing/ml-models/db16.4/polar-window20-lag0-resnet152-v2-tf/assets


INFO:tensorflow:Assets written to: /home/hwixley/Documents/4th-Year/Honours-Project/localhost-data-preprocessing/ml-models/db16.4/polar-window20-lag0-resnet152-v2-tf/assets


In [53]:
tflite_model_path

'/home/hwixley/Documents/4th-Year/Honours-Project/localhost-data-preprocessing/ml-models/db16.4/polar-window20-lag0-resnet152-v2-tf/saved_model.tflite'

In [59]:
with tf.device("/gpu:0"):
    model = tf.saved_model.load(tf_model_path)
    model.trainable = False

    input_tensor = tf.random.uniform([1, 1, 20, 75], dtype=tf.float32)
    out = model(**{'actual_input': input_tensor})

2022-04-16 14:37:00.623256: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-16 14:37:01.061805: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-04-16 14:38:16.855214: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-04-16 14:50:35.454916: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-04-16 14:50:35.550014: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-04-16 14:50:54.686921: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.27GiB with freed_by

KeyboardInterrupt: 

In [54]:
with tf.device("/gpu:0"):
    converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    tflite_model = converter.convert()

    # Save the model
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)

2022-04-16 13:48:12.257425: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:316] Ignored output_format.
2022-04-16 13:48:12.257443: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:319] Ignored drop_control_dependency.
2022-04-16 13:48:12.257446: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:325] Ignored change_concat_input_ranges.
2022-04-16 13:48:12.257552: I tensorflow/cc/saved_model/reader.cc:32] Reading SavedModel from: /home/hwixley/Documents/4th-Year/Honours-Project/localhost-data-preprocessing/ml-models/db16.4/polar-window20-lag0-resnet152-v2-tf/
2022-04-16 13:48:12.296505: I tensorflow/cc/saved_model/reader.cc:55] Reading meta graph with tags { serve }
2022-04-16 13:48:12.296529: I tensorflow/cc/saved_model/reader.cc:93] Reading SavedModel debug info (if present) from: /home/hwixley/Documents/4th-Year/Honours-Project/localhost-data-preprocessing/ml-models/db16.4/polar-window20-lag0-resnet152-v2-tf/
2022-04-16 13:48:1

In [40]:
tflite_model_path

'/home/hwixley/Documents/4th-Year/Honours-Project/localhost-data-preprocessing/ml-models/db16.4/polar-window20-lag0-resnet152-tf/polar-window20-lag0-resnet152.tflite'

In [ ]:
model

In [37]:
tf_model_path

'/home/hwixley/Documents/4th-Year/Honours-Project/localhost-data-preprocessing/ml-models/db16.4/polar-window20-lag0-resnet152-tf/'

In [38]:
TF_PATH = f"{tf_model_path}/saved_model.pb" # where the forzen graph is stored
TFLITE_PATH = f"{tf_model_path}/saved_model.tflite"
# protopuf needs your virtual environment to be explictly exported in the path
os.environ["PATH"] = "/opt/miniconda3/envs/convert/bin:/opt/miniconda3/bin:/usr/local/sbin:...."

# make a converter object from the saved tensorflow file
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(TF_PATH,  # TensorFlow freezegraph .pb model file
                                                      input_arrays=['input'], # name of input arrays as defined in torch.onnx.export function before.
                                                      output_arrays=['output'] # name of output arrays defined in torch.onnx.export function before.
                                                      )

# tell converter which type of optimization techniques to use
# to view the best option for optimization read documentation of tflite about optimization
# go to this link https://www.tensorflow.org/lite/guide/get_started#4_optimize_your_model_optional
# converter.optimizations = [tf.compat.v1.lite.Optimize.DEFAULT]

converter.experimental_new_converter = True

# I had to explicitly state the ops
converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]

tf_lite_model = converter.convert()
# Save the model.
with open(TFLITE_PATH, 'wb') as f:
    f.write(tf_lite_model)

2022-04-16 12:43:53.361022: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 12:43:53.361115: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.71GHz coreCount: 68 deviceMemorySize: 9.77GiB deviceMemoryBandwidth: 707.88GiB/s
2022-04-16 12:43:53.361135: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-04-16 12:43:53.361152: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-04-16 12:43:53.361159: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-04-16 12:43:53.361165: I tensorflow/stream_executor

Ignore 'tcmalloc: large alloc' warnings.


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x9b in position 3: invalid start byte